# Introduction to Recurrent Neural Networks

In [ ]:
using Knet

## A one layer MLP vs a simple RNN

([Elman 1990](http://onlinelibrary.wiley.com/doi/10.1207/s15516709cog1402_1/pdf)) A simple RNN takes the previous hidden state as an extra input, and returns the next hidden state as an output.

In [ ]:
# Comparison of a single MLP layer and corresponding RNN

function tanhmlp(param, input)
    weight,bias = param
    return tanh(input * weight .+ bias)
end

function tanhrnn(param, input, state)
    weight,bias = param
    return tanh(hcat(input, state) * weight .+ bias)
end;

<img src="images/rnn-vs-mlp.png" />(<a href="https://docs.google.com/drawings/d/1bPttFA0GEh7ti3xoWDma1ZbrQ21eulsPDeVUbPgdA7g/edit?usp=sharing">image source</a>)

## Architectures

<img src="images/diags.png" />
(<a href="http://karpathy.github.io/2015/05/21/rnn-effectiveness">image source</a>)

## Long Short-Term Memory (LSTM)
([Hochreiter and Schmidhuber, 1997](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf))
LSTM is a more sophisticated RNN module that performs better with long-range dependencies. 

<img src="images/LSTM3-chain.png" width=800 />
([image source](http://colah.github.io/posts/2015-08-Understanding-LSTMs))



$$\begin{align}
f_t &= \sigma(W_f\cdot[h_{t-1},x_t] + b_f) & \text{forget gate} \\
i_t &= \sigma(W_i\cdot[h_{t-1},x_t] + b_i) & \text{input gate} \\
\tilde{C}_t &= \tanh(W_C\cdot[h_{t-1},x_t] + b_C) & \text{cell candidate} \\
C_t &= f_t \ast C_{t-1} + i_t \ast \tilde{C}_t & \text{new cell} \\
o_t &= \sigma(W_o\cdot[h_{t-1},x_t] + b_o) & \text{output gate} \\
h_t &= o_t \ast \tanh(C_t) & \text{new output}\\
\end{align}$$

## A pure Julia LSTM implementation

In [ ]:
function lstm(param, input, state)
    weight,bias = param
    hidden,cell = state
    h       = size(hidden,2)
    gates   = hcat(input,hidden) * weight .+ bias
    forget  = sigm.(gates[:,1:h])
    ingate  = sigm.(gates[:,1+h:2h])
    outgate = sigm.(gates[:,1+2h:3h])
    change  = tanh.(gates[:,1+3h:4h])
    cell    = cell .* forget + ingate .* change
    hidden  = outgate .* tanh.(cell)
    return (hidden,cell)
end;

## Gated Recurrent Unit (GRU)

Introduced by <a href="http://arxiv.org/pdf/1406.1078v3.pdf">Cho, et al. (2014)</a>, GRU combines the forget and input gates into a single “update gate.”

<img src="images/LSTM3-var-GRU.png" />
(<a href="http://colah.github.io/posts/2015-08-Understanding-LSTMs/">image source</a>)



## A more efficient cuDNN implementation: rnninit and rnnforw

In [ ]:
# Sample usage
rnnSpec,rnnWeights = rnninit(inputSize, hiddenSize; rnnType=:gru)
rnnOutput = rnnforw(rnnSpec, rnnWeights, rnnInput)[1] # note the [1] at the end, rnnforw returns a tuple

In [ ]:
@doc rnninit

In [ ]:
@doc rnnforw